In [1]:
import pandas as pd
import numpy as np
import random
import time
from haplotypeSegmentGraph import haplotypeSegmentGraph
from haplotypeHMM import haplotypeHMM
from calculateSwitchErrorRate import calculate_switch_error_rate

In [2]:
hap_f_path = 'examples/CCDG_14151_B01_GRM_WGS_2020-08-05_chr19.filtered.shapeit2-duohmm-phased.2504.43-47Mb.ALL.maf01.haps.gz'

In [3]:
total_numSNPs = 26246
total_numSamples = 2504
B = 3

In [179]:
col_seeds = []
row_seeds = []
for examined_numSNPs in [5,10,20,50,100,200]:
    col_seeds.append(random.randint(0,1000))
for num_samples in [100,300,600,1200,2504]:
    row_seeds.append(random.randint(0,1000))

In [5]:
# col_seeds = [877, 927, 437, 264, 910, 656, 425, 252, 864, 150, 289, 952, 756, 485, 852, 471, 927, 956, 397, 815, 112, 387, 722, 732, 113, 57, 207, 925, 720, 861, 869, 751, 112, 211, 760]
# row_seeds = [994, 302, 123, 843, 740, 285, 250, 150, 431, 495, 148, 478, 524, 434, 534, 827, 184, 788, 10, 196, 460, 311, 340, 607, 762, 725, 794, 988, 589, 471, 92, 335, 410, 460, 601]

In [181]:
threads = 20

In [8]:
df = pd.read_csv(hap_f_path,header=None,sep=' ')
# make sure sort by position
df = df.sort_values(by=2)

In [ ]:
time_dict = {}
results_dict = {}
genos_dict = {}
labels_dict = {}
for B in [2,3,4,5,6]:
    time_dict[B] = {}
    results_dict[B] = {}
    genos_dict[B] = {}
    labels_dict[B] = {}
    # SER_dict = {}
    for snp_index,examined_numSNPs in enumerate([5,10,20,50,100,200]):
        if examined_numSNPs in [5,10]:
            threads = 6
        else:
            threads = 20
        time_dict[B][examined_numSNPs] = {}
        results_dict[B][examined_numSNPs] = {}
        genos_dict[B][examined_numSNPs] = {}
        labels_dict[B][examined_numSNPs] = {}
        for sample_index,num_samples in enumerate([100,300,600,1200,2504]):
            # set random seed
            np.random.seed(row_seeds[sample_index])
            snps_masking = np.random.choice(range(df.shape[0]), examined_numSNPs)
            snps_df = df.loc[snps_masking]
            snps_df = snps_df.sort_values(by=2)
            # prepare input data
            genetic_pos = snps_df[2]
            haps = snps_df.loc[:,5:].values
            labels = []
            for i in range(haps.shape[1]//2):
                labels.append((haps[:,i],haps[:,haps.shape[1]//2+i]))
            labels = np.array(labels)
            genos = np.full(shape=(haps.shape[0],haps.shape[1]//2),fill_value=-1,dtype=int)
            ## create genotypes by combining each pair of haplotypes
            for i in range(genos.shape[1]):
                genos[:,i] = haps[:,2*i] + haps[:,2*i+1]
            genos = genos.T
            # set the seed for col
            np.random.seed(col_seeds[snp_index])
            sample_masking = np.random.choice(range(genos.shape[0]), num_samples)
            genos = genos[sample_masking,:]
            labels = labels[sample_masking,:,:]
            # run the HMM
            st = time.time()
            hap_graph = haplotypeSegmentGraph(genos,genetic_pos,B)
            hmm = haplotypeHMM(hap_graph)
            results = hmm.predict(genos,threads=threads)
            duration = time.time()  - st
            # evaluation
    #         all_SER = calculate_switch_error_rate(results,labels)
    #         SER_dict[examined_numSNPs][num_samples]  = all_SER
            time_dict[B][examined_numSNPs][num_samples]  = duration * threads
            results_dict[B][examined_numSNPs][num_samples] = results
            genos_dict[B][examined_numSNPs][num_samples] = genos
            labels_dict[B][examined_numSNPs][num_samples] = labels
    with open(f'results_B_{B}','wb') as f:
        pickle.dump([time_dict,results_dict,genos_dict,labels_dict],f)


In [77]:
# import pickle

In [78]:
# with open('results.pkl','wb') as f:
#     pickle.dump([time_dict,results_dict,genos_dict,labels_dict],f)

In [128]:
def get_switch_error(prediction,truth):
    SERs = []
#     hetero_masking = truth[0] + truth[1] == 1
    hetero_masking = truth[0] + truth[1] != -1
    hetero_prediction = prediction[0][hetero_masking]
    hetero_truth = truth[0][hetero_masking]
    if hetero_truth.shape[0] == 0:
        return 0
    hap_type = hetero_prediction == hetero_truth
    num_switch = 0
    for i in range(len(hap_type)-1):
        if hap_type[i] != hap_type[i+1]:
            num_switch += 1
    SERs.append(num_switch/hap_type.shape[0])
    
    hetero_prediction = prediction[1][hetero_masking]
    hetero_truth = truth[0][hetero_masking]
    if hetero_truth.shape[0] == 0:
        return 0
    hap_type = hetero_prediction == hetero_truth
    num_switch = 0
    for i in range(len(hap_type)-1):
        if hap_type[i] != hap_type[i+1]:
            num_switch += 1
    SERs.append(num_switch/hap_type.shape[0])
    
    return np.min(SERs)
def calculate_switch_error_rate(results,labels):
    all_SER = []
    for i in range(len(results)):
        SER = get_switch_error(results[i],labels[i])
        all_SER.append(SER)
    return all_SER

In [131]:
from scipy.spatial.distance import hamming

In [144]:
def calculcate_hamming_dist(results,labels):
    all_error = []
    for i in range(len(results)):
        h1 = hamming(results[i][0],labels[i][0]) + hamming(results[i][1],labels[i][1])
        h2 = hamming(results[i][0],labels[i][1]) + hamming(results[i][1],labels[i][0])
        all_error.append(np.min([h1,h2]))
    return all_error

In [129]:
x = []
y = []
num_SNPs = 200
for sample_size in results_dict[num_SNPs]:
    x.append(sample_size)
    y.append(np.mean(calculate_switch_error_rate(results_dict[num_SNPs][sample_size],labels_dict[num_SNPs][sample_size])))

In [158]:
x = []
y = []
sample_size = 2504
for examined_numSNPs in [5,10,20,50,100,200]:
#     for sample_size in results_dict[examined_numSNPs]:
    x.append(examined_numSNPs)
    y.append(np.mean(calculate_switch_error_rate(results_dict[examined_numSNPs][sample_size],labels_dict[examined_numSNPs][sample_size])))

In [159]:
y

[0.2930511182108626,
 0.3047923322683706,
 0.2319888178913738,
 0.23825079872204474,
 0.26673322683706074,
 0.29781349840255594]

In [156]:
x = []
y = []
sample_size = 2504
for examined_numSNPs in [5,10,20,50,100,200]:
#     for sample_size in results_dict[examined_numSNPs]:
    x.append(examined_numSNPs)
    y.append(np.mean(calculcate_hamming_dist(results_dict[examined_numSNPs][sample_size],labels_dict[examined_numSNPs][sample_size])))

6.34

In [171]:
# x = []
# y = []
# for examined_numSNPs in [5,10,20,50,100,200]:
#     for num_samples in [100,300,600,1200,2504]:
#         hap_graph = haplotypeSegmentGraph(genos_dict[examined_numSNPs][num_samples],genetic_pos,B)
#         SER = np.mean(calculate_switch_error_rate(results_dict[examined_numSNPs][num_samples],labels_dict[examined_numSNPs][num_samples]))
#         state = sum([len(nodes) for nodes in hap_graph.nodes.values()]) / len(hap_graph.nodes)
#         x.append(state)
#         y.append(SER)